# Volatility indicators

The purpose of this notebook is to compute and plot different volatility indicators

In [43]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.offline as pyo
import plotly.graph_objects as go
import plotly.express as px
from market_analysis.dataset import get_stock_data
from market_analysis.features import get_Nday_return
from loguru import logger
logger.disable("market_analysis")

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = "plotly"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
df = get_stock_data('AZN.L', end_date=(2024, 12, 6), days=365.25*12)
FTSE = get_stock_data('FTSE', end_date=(2024, 12, 6), days=365.25*12)

[*********************100%***********************]  1 of 1 completed


### Compute & plot some volatility indicators

In [115]:
log_returns = get_Nday_return(df, days=1, log=True)

### 1) Trailing volatility

Standard deviation of log returns over a given period

In [117]:
LAG = 20
volatility = log_returns.rolling(window=LAG).std()*np.sqrt(LAG)
volatility.plot().update_layout(autosize=False, width=800, height=500)


### 2) Sharpe ratio

Risk-adjusted return: average return earned in excess of risk-free rate / volatility

In [118]:
Rf = 0.01/252 # annual risk free rate of return / trading days

sharpe_ratio = (log_returns.rolling(window=LAG).mean() - Rf)*LAG/volatility
sharpe_ratio.plot().update_layout(autosize=False, width=800, height=500)

### 3) Sortino Ratio

Same as above, but only considers downwards deviations when computing volatility

In [119]:
LAG = 20
sortino_vol = log_returns[log_returns < 0].rolling(window=LAG).std()*np.sqrt(LAG)
sortino_ratio =  (log_returns.rolling(window=LAG).mean() - Rf)*LAG/sortino_vol
sharpe_ratio.plot().update_layout(autosize=False, width=800, height=500)

### 4) Modigliani ratio (M2 ratio) 

Return of a portfolio, adjusted for the risk of the portfolio relative to that of some benchmark

In [120]:
FTSE_returns = get_Nday_return(FTSE, days=1, log=True)
FTSE_volatility = log_returns.rolling(window=LAG).std()*np.sqrt(LAG)

M2_ratio = (sharpe_ratio*FTSE_volatility/LAG + Rf)*LAG
sharpe_ratio.plot().update_layout(autosize=False, width=800, height=500)